# Server Monitoring

The script below provides guidance on how to conduct server monitoring using CSVs as a data source or using a database as a data source

## CSV Integration

In [ ]:
from datetime import datetime
from subprocess import call

import pandas as pd
import psutil
import numpy as np

In [ ]:
def RecordMetrics():

    drps = psutil.disk_partitions()
    drives = [dp.device for dp in drps if dp.fstype == 'NTFS']
    
    df_csv = pd.read_csv('ServerMonitoring.csv')
    
    history_id = max(df_csv['HistoryID']) + 1    

    df = pd.DataFrame(
        {
            'CPU': [psutil.cpu_percent()],
            'VirtualMemory': [psutil.virtual_memory()[2]],
            'HistoryID' : [history_id],
            'LastBootedAt' : datetime.fromtimestamp(psutil.boot_time()).strftime('%Y-%m-%d %H:%M:%S')
            
        })

    for drive in drives:
        df['{}_DriveUsage'.format(drive.replace(":\\",""))] = psutil.disk_usage(drive)[3]

    df['LoadDate'] = datetime.now()
    
    df = df.append(df_csv)
    df.to_csv('ServerMonitoring.csv', index=False)

## Database Integration

In [ ]:
from sqlalchemy import create_engine, event
from datetime import datetime
from subprocess import call

import urllib
import pandas as pd
import psutil
import pyodbc
import pymsteams
import numpy as np

driver = '{SQL Server}'
server = ''
user = ''
password = ''
database = ''
table = 'ServerMonitoring'

params = urllib.parse.quote_plus(r'DRIVER={};SERVER={};DATABASE={};UID={};PWD={}'.format(driver, server, database, user, password))
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)

### Write Metrics to Database

In [ ]:
def RecordMetrics():

    drps = psutil.disk_partitions()
    drives = [dp.device for dp in drps if dp.fstype == 'NTFS']

    df = pd.DataFrame(
        {
            'CPU': [psutil.cpu_percent()],
            'VirtualMemory': [psutil.virtual_memory()[2]],
            'LastBootedAt' : datetime.fromtimestamp(psutil.boot_time()).strftime('%Y-%m-%d %H:%M:%S')
        })

    for drive in drives:
        df['{}_DriveUsage'.format(drive.replace(":\\",""))] = psutil.disk_usage(drive)[3]

    df['LoadDate'] = datetime.now()
    
    #if_exists="replace" if the table does not yet exist, then add HistoryID (or ID) as the auto-incremented primary key
    df.to_sql(table, engine, if_exists="append", index=False)

### # Monitor CPU Average

In [ ]:
def CheckCPU():
    
    df_CPU = df.read_sql('SELECT TOP(5) CPU FROM dbo.ServerMonitoring ORDER BY LoadDate DESC', conn_str)
    
    avg_CPU = df_CPU['CPU'].mean()
    
    # if the CPU Average is above 90%
    if avg_CPU > 90:
        
        print('High CPU Usage Detected: {}%'.format(round(avg_CPU,2)))        

### Monitor Boot Time

In [ ]:
def CheckBootTime():
    
    df_CPU = df.read_sql('SELECT TOP(1) LastBootedAt FROM dbo.ServerMonitoring ORDER BY LoadDate DESC', conn_str)
    
    current_timestamp = datetime.now()
    
    up_time = current_timestamp - boot_time
    
    days = str(up_time).split(" days, ")[0]
    time = str(up_time).split(" days, ")[1]

    hours = time.split(':')[0]
    mins = time.split(':')[1]
    secs = time.split(':')[2]

    # if PC last rebooted in the last 9 hours
    if days == 0 and hours < 10:

        print('Server was rebooted {} hours and {} minutes ago'.format(hours, minutes))

### Executing Metrics Collection

In [ ]:
RecordMetrics()

### Execute Monitoring

In [ ]:
CheckCPU()
CheckBootTime()